## PGA Draftkings Notebook
Use [PGA Website](https://www.pgatour.com/tournaments/schedule.html) to look up tournament info and fill out first USER INPUT block below. (Fetch/XHR from Network tab, Paylod sub-tab)

Looks like the tournament ID is also in the address bar.

### User Input

In [1]:
# === USER INPUTS ===
# Old Tournament
old_tournament_name = "The RSM Classic"
tournament_date = "11/23/2025"  # Ending date of tournament
old_course = "Sea Island Resort"
tournament_id = "R2025493"  # Tournament ID from the PGA Tour API - Also visible in the URL of the tournament page

# New Tournament
new_tournament_name = "Hero World Challenge"
new_ending_date = "12/7/2025"
new_course = "Albany GC"
new_season = 2025


# === LIBRARIES AND VARIABLES ===
# Import necessary libraries
import requests
import pandas as pd
from datetime import datetime
import sqlite3 as sql
import numpy as np
from numpy import nan
import os
import importlib
import utils.db_utils
from utils.db_utils import TOURNAMENT_NAME_MAP, PLAYER_NAME_MAP

# === TOURNAMENT CONFIG ===
tournament_config = {
    "old": {
        "name": old_tournament_name,
        "date": tournament_date,
        "course": old_course,
        "id": tournament_id
    },
    "new": {
        "name": new_tournament_name,
        "course": new_course,
        "season": new_season,
        "ending_date": pd.to_datetime(new_ending_date, format="%m/%d/%Y"),
        "quoted_course": f'"{new_course}"',
        "quoted_name": f"'{new_tournament_name}'"
    }
}

this_week_key = str(tournament_config["new"]["ending_date"].date())

# Create a minimal one-row history DataFrame
this_week_history = pd.DataFrame([{
    "SEASON": tournament_config["new"]["season"],
    "TOURNAMENT": tournament_config["new"]["name"],
    "ENDING_DATE": tournament_config["new"]["ending_date"],
    "COURSE": tournament_config["new"]["course"],
    "TOURN_ID": None
}])

### Update Database

#### Old Tournament

In [2]:
importlib.reload(utils.db_utils)  # Only needed if you're actively editing db_utils.py
from utils.db_utils import update_tournament_results

# Change these each year!!
season = 2025
year = 20250  # Unique GraphQL year distinguishing number in case of multiple per year

# Run the update
db_path = "data/golf.db"  # Or use os.path.join("data", "golf.db")
tournDf = update_tournament_results(tournament_config, db_path, season, year)

# Show just the most recent tournament added for confirmation
from sqlalchemy import create_engine

engine = create_engine(f"sqlite:///{db_path}")

query = f"""
SELECT *
FROM tournaments
WHERE TOURN_ID = '{tournament_config['old']['id']}'
  AND ENDING_DATE = '{datetime.strptime(tournament_config['old']['date'], '%m/%d/%Y').date()}'
"""

recent = pd.read_sql(query, engine)
engine.dispose()
recent.sort_values(by='FINAL_POS').head()

📦 Fetching results for tournament ID R2025493 (The RSM Classic), year: 20250
✅ 156 new rows added for 'The RSM Classic'


,SEASON,ENDING_DATE,TOURN_ID,TOURNAMENT,COURSE,PLAYER,POS,FINAL_POS,ROUNDS:1,ROUNDS:2,ROUNDS:3,ROUNDS:4,OFFICIAL_MONEY,FEDEX_CUP_POINTS
0,2025,2025-11-23,R2025493,The RSM Classic,Sea Island Resort,Sami Valimaki,1,1,-6,-8,-5,-4,"$1,260,000.00",500.000
1,2025,2025-11-23,R2025493,The RSM Classic,Sea Island Resort,Max McGreevy,2,2,-8,-3,-4,-7,"$763,000.00",300.000
2,2025,2025-11-23,R2025493,The RSM Classic,Sea Island Resort,Ricky Castillo,3,3,-7,-5,-1,-8,"$483,000.00",190.000
3,2025,2025-11-23,R2025493,The RSM Classic,Sea Island Resort,Nico Echavarria,T4,4,-6,-5,-3,-5,"$294,583.33",115.000
4,2025,2025-11-23,R2025493,The RSM Classic,Sea Island Resort,Lee Hodges,T4,4,-4,-7,-4,-4,"$294,583.33",115.000


#### Stats

In [3]:
importlib.reload(utils.db_utils)
from utils.db_utils import update_season_stats  # <- This line is essential

# Change these each year!!
statsYear = 2025

stats_df = update_season_stats(statsYear, db_path)
stats_df.head()

✅ Overwrote stats for season 2025 with 998 rows.


,PLAYER,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,...,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,SEASON
0,A.J. Ewart,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,491,.2945,2025
1,Aaron Baddeley,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,704,.1755,2025
2,Aaron Cockerill,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,437,.3443,2025
3,Aaron Rai,12.0,0.942,22.0,0.391,21.0,0.476,68.0,0.077,132.0,...,289.6,2.0,73.85%,18.0,70.52%,101.0,59.16%,23,3.1098,2025
4,Abel Gallegos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,991,.0919,2025


#### Odds
Not usually needed for weekly routine.

**Manual Fix! Odds name cleanup (only needed when joins fail)**

Make sure to update the dictionaries in db_utils.py if new names need to be added.

In [17]:
importlib.reload(utils.db_utils)
from utils.db_utils import clean_odds_names, PLAYER_NAME_MAP, TOURNAMENT_NAME_MAP

db_path = "data/golf.db" 
updated_odds = clean_odds_names(db_path, TOURNAMENT_NAME_MAP, PLAYER_NAME_MAP)
updated_odds.head()

✅ Cleaned and updated 516 rows in 'odds' table.


,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS,TOURNAMENT_ORIG,PLAYER_ORIG
8275,2024,World Wide Technology Championship,2024-11-10,Max Greyserman,16/1,16.0,World Wide Tech Champ,Max Greyserman
8276,2024,World Wide Technology Championship,2024-11-10,Doug Ghim,18/1,18.0,World Wide Tech Champ,Doug Ghim
8277,2024,World Wide Technology Championship,2024-11-10,J.J. Spaun,25/1,25.0,World Wide Tech Champ,J.J. Spaun
8278,2024,World Wide Technology Championship,2024-11-10,Beau Hossler,25/1,25.0,World Wide Tech Champ,Beau Hossler
8279,2024,World Wide Technology Championship,2024-11-10,Ben Griffin,25/1,25.0,World Wide Tech Champ,Ben Griffin


**Historical Odds Updates**

Only run this when loading in entire year odds at the start of each year or if corrections need to be made.  This will load in the entire year into the database and update it using the dictionary in db_utils.py.

In [15]:
importlib.reload(utils.db_utils)
from utils.db_utils import import_historical_odds

oddsYear = "2022-2023"    # URL segment
season = 2023             # PGA Tour season
db_path = "data/golf.db"

odds_df = import_historical_odds(oddsYear, season, db_path)
odds_df.head()

✅ Inserting 120 new rows into odds table...


,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS
4,2023,Fortinet Championship,2022-09-18,Hideki Matsuyama,18/1,18.0
5,2023,Fortinet Championship,2022-09-18,Max Homa,18/1,18.0
6,2023,Fortinet Championship,2022-09-18,Corey Conners,20/1,20.0
7,2023,Fortinet Championship,2022-09-18,Maverick McNealy,25/1,25.0
8,2023,Fortinet Championship,2022-09-18,Taylor Pendrith,25/1,25.0


**Not normally needed**

The code below is a way to troubleshoot the odds function that is in db_utils.py.  If it does not pull the stats correctly, we can run it outside of the db_utils.py file and explore what might be wrong with it, iterating until it works, and then using that fix to update db_utils.py.  This is not normally needed, but wanted to leave this in place as a reference because this odds website can be tricky.

In [16]:
import pandas as pd
import numpy as np
import requests
import re
from datetime import datetime
from io import StringIO

# === USER INPUT ===
oddsYear = "2020-2021"    # URL segment
season = 2021        # PGA Tour season

url = f"http://golfodds.com/archives-{oddsYear}.html"
response = requests.get(url)
tables = pd.read_html(StringIO(response.text))
# raw_df = tables[5]  # the actual table of interest
# Find the largest 2-column table that contains at least some odds-like strings
raw_df = None
for tbl in tables:
    if tbl.shape[1] == 2 and tbl.shape[0] > 50:  # Rough filter
        sample = tbl.iloc[:, 1].astype(str).str.contains(r"\d+/\d+").sum()
        if sample > 5:
            raw_df = tbl
            break

if raw_df is None:
    raise ValueError("❌ Could not find valid odds table on the page.")

# === STEP 1: Initial clean-up ===
df = raw_df.dropna(how="all").reset_index(drop=True)
df.columns = ["PLAYER", "ODDS"]

# 🔧 Clean up non-breaking spaces and extra whitespace
df["PLAYER"] = (
    df["PLAYER"]
    .astype(str)
    .str.replace("\xa0", " ", regex=False)
    .str.replace(r"\s+", " ", regex=True)
    .str.strip()
)

df.insert(loc=0, column="SEASON", value=season)
df.insert(loc=1, column="TOURNAMENT", value=np.nan)
df.insert(loc=2, column="ENDING_DATE", value=np.nan)

# === STEP 2: Helper function for parsing date strings ===
def parse_ending_date(text):
    import re
    from datetime import datetime

    # Normalize whitespace and symbols
    text = (
        text.replace("\u2013", "-")
            .replace("–", "-")
            .replace("\xa0", " ")
    )
    text = re.sub(r"\bSept(?!ember)\b", "Sep", text)

    # ✅ Fix typo: "Match" → "March" only when used in a date context
    text = re.sub(r"\bMatch(?=\s+\d{1,2}\s*[-–]\s*\d{1,2},\s*\d{4})", "March", text)

    # Pattern 1: "July 30 - August 2, 2015" or "Oct 29 - Nov 1, 2015"
    match = re.search(r"(\w+)\s\d+\s*-\s*(\w+)\s(\d+),\s(\d{4})", text)
    if match:
        month2, day2, year = match.group(2), match.group(3), match.group(4)
        for fmt in ["%B %d, %Y", "%b %d, %Y"]:
            try:
                return datetime.strptime(f"{month2} {day2}, {year}", fmt).date()
            except ValueError:
                continue

    # Pattern 2: "November 21-24, 2024"
    match = re.search(r"(\w+)\s\d+-\d+,\s(\d{4})", text)
    if match:
        month, year = match.group(1), match.group(2)
        day = re.search(r"(\d+)-(\d+)", text).group(2)
        for fmt in ["%B %d, %Y", "%b %d, %Y"]:
            try:
                return datetime.strptime(f"{month} {day}, {year}", fmt).date()
            except ValueError:
                continue

    # Pattern 3: "Sunday, October 20, 2019"
    try:
        return datetime.strptime(text.strip(), "%A, %B %d, %Y").date()
    except ValueError:
        pass

    # Pattern 4: "October 20, 2019"
    try:
        return datetime.strptime(text.strip(), "%B %d, %Y").date()
    except ValueError:
        pass

    return None

last_tourn_name = None
last_date = None

# === STEP 3: Iterate block by block ===
final_rows = []
i = 0
last_tourn_name = None
last_end_date = None

while i < len(df) - 4:
    player_i = str(df.loc[i, "PLAYER"])
    player_i2 = str(df.loc[i + 2, "PLAYER"])
    player_i3 = str(df.loc[i + 3, "PLAYER"]).lower()

    # Detect start of a new tournament block
    is_header = (
        pd.isna(df.loc[i, "ODDS"]) and
        pd.isna(df.loc[i + 1, "ODDS"]) and (
            re.search(r"\w+\s\d+\s*[-–]\s*(\w+\s)?\d+,\s\d{4}", player_i2) or
            re.search(r"(Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday),?\s+\w+\s\d{1,2},\s\d{4}", player_i2)
        )
    )

    if is_header:
        tourn_name = player_i.strip()
        end_date = parse_ending_date(player_i2)

        # Skip cancelled or empty blocks
        if "cancelled" in player_i3:
            print(f"⚠️ Skipping cancelled tournament: {tourn_name} — {end_date}")
            i += 4
            continue

        # Avoid duplicate block processing
        if tourn_name == last_tourn_name and end_date == last_end_date:
            i += 1
            continue

        print(f"📍 Detected: {tourn_name} — Ending: {end_date}")
        last_tourn_name = tourn_name
        last_end_date = end_date
        i += 4  # Skip header lines

        # Collect all player rows until next header block
        while i < len(df) - 2:
            next_i2 = str(df.loc[i + 2, "PLAYER"])
            is_next_header = (
                pd.isna(df.loc[i, "ODDS"]) and
                pd.isna(df.loc[i + 1, "ODDS"]) and (
                    re.search(r"\w+\s\d+\s*[-–]\s*(\w+\s)?\d+,\s\d{4}", next_i2) or
                    re.search(r"(Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday),?\s+\w+\s\d{1,2},\s\d{4}", next_i2)
                )
            )
            if is_next_header:
                break

            if pd.notna(df.loc[i, "ODDS"]):
                row = df.loc[i].copy()
                row["TOURNAMENT"] = tourn_name
                row["ENDING_DATE"] = end_date
                final_rows.append(row)
            i += 1
    else:
        i += 1

# === STEP 4: Create cleaned DataFrame ===
clean_df = pd.DataFrame(final_rows)

# ✅ Prevent crash if nothing was parsed
if clean_df.empty or "PLAYER" not in clean_df.columns:
    print(f"⚠️ No valid tournament blocks detected for season {season} ({oddsYear})")
    final_df = pd.DataFrame()  # Safe fallback
else:
    # Remove winner tag
    clean_df["PLAYER"] = clean_df["PLAYER"].str.replace(r"\s\*Winner\*", "", regex=True)

    # Clean odds to numeric
    clean_df["VEGAS_ODDS"] = (
        clean_df["ODDS"]
        .str.replace(",", "")
        .str.extract(r"(\d+)/(\d+)")
        .astype(float)
        .apply(lambda x: x[0] / x[1], axis=1)
    )

    # Final output with source index for debugging
    final_df = clean_df[
        ["SEASON", "TOURNAMENT", "ENDING_DATE", "PLAYER", "ODDS", "VEGAS_ODDS"]
    ].reset_index(drop=True)

    # Drop non-standard team events (e.g., Presidents Cup, Ryder Cup)
    drop_terms = ["Presidents Cup", "Ryder Cup"]
    final_df = final_df[~final_df["TOURNAMENT"].str.contains("|".join(drop_terms), case=False, na=False)]

    display(final_df.head())



📍 Detected: Safeway Open — Ending: 2020-09-13
📍 Detected: US Open — Ending: 2020-09-20
📍 Detected: R & C Championship — Ending: 2020-09-27
📍 Detected: at Big Cedar Lodge - — Ending: 2020-09-22
📍 Detected: Sanderson Farms Champ — Ending: 2020-10-04
📍 Detected: Shriners H for C Open — Ending: 2020-10-11
📍 Detected: The CJ Cup — Ending: 2020-10-18
📍 Detected: ZOZO CHAMPIONSHIP — Ending: 2020-10-25
📍 Detected: Bermuda Championship — Ending: 2020-11-01
📍 Detected: Vivint Houston Open — Ending: 2020-11-08
📍 Detected: The Masters — Ending: 2020-11-15
📍 Detected: The RSM Classic — Ending: 2020-11-22
📍 Detected: Champions for Change — Ending: 2020-11-27
📍 Detected: Mayakoba Golf Classic — Ending: 2020-12-06
📍 Detected: QBE Shootout — Ending: 2020-12-13
📍 Detected: Sentry Tourn of Champions — Ending: 2021-01-10
📍 Detected: Sony Open in Hawaii — Ending: 2021-01-17
📍 Detected: The American Express — Ending: 2021-01-24
📍 Detected: Abu Dhabi HSBC Champ — Ending: 2021-01-24
📍 Detected: Farmers Insura

,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS
0,2021,Safeway Open,2020-09-13,Phil Mickelson,20/1,20.0
1,2021,Safeway Open,2020-09-13,Si Woo Kim,20/1,20.0
2,2021,Safeway Open,2020-09-13,Brendan Steele,20/1,20.0
3,2021,Safeway Open,2020-09-13,Shane Lowry,25/1,25.0
4,2021,Safeway Open,2020-09-13,Sergio Garcia,30/1,30.0


In [17]:
from datetime import datetime, date
# ✅ Check for non-date types in ENDING_DATE
non_dates = final_df[~final_df["ENDING_DATE"].apply(lambda x: isinstance(x, date))]

print(f"🧪 Rows with invalid ENDING_DATE values: {len(non_dates)}")
display(non_dates.head(10))



🧪 Rows with invalid ENDING_DATE values: 0


,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS


In [18]:
dupes = final_df.duplicated(subset=["SEASON", "TOURNAMENT", "ENDING_DATE", "PLAYER"], keep=False)

print(f"🚨 Duplicate primary keys in final_df: {dupes.sum()}")
display(final_df[dupes].sort_values(by=["SEASON", "TOURNAMENT", "PLAYER"]))

🚨 Duplicate primary keys in final_df: 0


,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS


#### Misc. Cleaning
Not normally needed

In [26]:
importlib.reload(utils.db_utils)
from utils.db_utils import clean_player_names_in_table, PLAYER_NAME_MAP

db_path = "data/golf.db"

# Run for all relevant tables
clean_player_names_in_table(db_path, "tournaments", PLAYER_NAME_MAP)
clean_player_names_in_table(db_path, "stats", PLAYER_NAME_MAP)
clean_player_names_in_table(db_path, "odds", PLAYER_NAME_MAP)

ℹ️ No player names needed updates in 'tournaments'.
ℹ️ No player names needed updates in 'stats'.
ℹ️ No player names needed updates in 'odds'.


,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS,PLAYER_ORIG


## Historical Data

### Pull Relevant Seasons
Do a check to see when this course or tournament have been historically played.

In [5]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_combined_history_seasons

# === USER INPUT ===
seasons = list(range(2016, 2025))  # Adjust as needed
db_path = "data/golf.db"

# Pull course and tournament from config
n_course = tournament_config["new"]["course"]
n_tourn = tournament_config["new"]["name"]

# Fetch relevant history
history_df = get_combined_history_seasons(db_path, course=n_course, tournament=n_tourn, allowed_seasons=seasons)
history_df.head(20)


ℹ️ Found 2 relevant tournaments from course or tournament name.


,SEASON,COURSE,TOURN_ID,TOURNAMENT,ENDING_DATE
0,2023,Albany GC,478,Hero World Challenge,2022-12-04
40,2024,Albany GC,R2024478,Hero World Challenge,2024-12-08


### Cut Percentage and FedEx Points
Use a rolling-window approach to look at the most recent cut percentage and how many FedEx cup points have been accumulated recently. This will intentionally not match the PGA Tour stats that start over every year, but will have the same amount of data all the time.  We also add a new feature called Form Density which divides the FedEx Cup Points by the Total Events.

In [6]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_cut_and_fedex_history

cuts = get_cut_and_fedex_history("data/golf.db", history_df, window_months=9)
# cuts["2024-05-12"].head(20)

for end_date, df in cuts.items():
    print(f"\n📆 {end_date} — {df['TOURNAMENT'].iloc[0]} ({len(df)} players)")
    display(df.head(3))


📆 2022-12-04 — Hero World Challenge (603 players)


,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,A.J. Ewart,1,0,0.00,0.0,0.00,0,2022-12-04,Hero World Challenge
1,Aaron Baddeley,10,6,180.44,60.0,18.04,1,2022-12-04,Hero World Challenge
2,Aaron Cockerill,1,1,0.00,100.0,0.00,1,2022-12-04,Hero World Challenge



📆 2024-12-08 — Hero World Challenge (579 players)


,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,Aaron Baddeley,16,9,94.033,56.2,5.88,0,2024-12-08,Hero World Challenge
1,Aaron Cockerill,2,0,0.000,0.0,0.00,0,2024-12-08,Hero World Challenge
2,Aaron Rai,16,15,1698.132,93.8,106.13,14,2024-12-08,Hero World Challenge


### Recent Form

In [7]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_recent_avg_finish

recent_form = get_recent_avg_finish("data/golf.db", history_df, window_months=9)

# Example preview
for date, df in recent_form.items(): 
    print(f"\n📆 {date} — {df['TOURNAMENT'].iloc[0]} ({len(df)} players)")
    display(df.head(3))


📆 2022-12-04 — Hero World Challenge (603 players)


,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Ryo Hisatsune,1,12.0,17.31,2022-12-04,Hero World Challenge
1,Anthony Quayle,1,15.0,21.64,2022-12-04,Hero World Challenge
2,Mikumu Horikawa,1,16.0,23.08,2022-12-04,Hero World Challenge



📆 2024-12-08 — Hero World Challenge (579 players)


,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Scottie Scheffler,13,5.7,2.16,2024-12-08,Hero World Challenge
1,Yuta Sugiura,1,6.0,8.66,2024-12-08,Hero World Challenge
2,Taiga Semikawa,1,9.0,12.98,2024-12-08,Hero World Challenge


### Course History

In [8]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_course_history

# Filter history_df for only the course we're targeting
target_course = tournament_config["new"]["course"]
course_df = history_df[history_df["COURSE"] == target_course]
course_hist = get_course_history("data/golf.db", course_df)

# View example
for date, df in course_hist.items():
    if not df.empty:
        print(f"\n🏌️‍♂️ Course history for {df['TOURNAMENT'].iloc[0]} on {date}")
        display(df.head(3))


🏌️‍♂️ Course history for Hero World Challenge on 2024-12-08


,PLAYER,TOTAL_EVENTS_PLAYED,COURSE_HISTORY,adj_ch,ENDING_DATE,COURSE,TOURNAMENT
0,Billy Horschel,1,13.0,18.76,2024-12-08,Albany GC,Hero World Challenge
1,Brian Harman,1,8.0,11.54,2024-12-08,Albany GC,Hero World Challenge
2,Cameron Young,2,9.0,8.19,2024-12-08,Albany GC,Hero World Challenge


In [9]:
#  DEBUGGING: Check for empty DataFrames or missing columns
for date_key, df in course_hist.items():
    if df.empty:
        print(f"ℹ️ Empty course_hist for date {date_key}")
    elif "PLAYER" not in df.columns:
        print(f"❌ Missing 'PLAYER' column in course_hist[{date_key}]. Columns present: {df.columns.tolist()}")
        display(df.head())

ℹ️ Empty course_hist for date 2022-12-04


## Training Dataset

In [10]:
pd.set_option("display.max_columns", None)   # Show all columns
importlib.reload(utils.db_utils)
from utils.db_utils import build_training_rows
training_df = build_training_rows(
    db_path,
    history_df,
    cuts,
    recent_form,
    course_hist,
)
training_df.head(10)
# training_df.info()
# training_df.isna().sum().sort_values(ascending=False)

,SEASON,ENDING_DATE,TOURNAMENT,COURSE,PLAYER,POS,FINAL_POS,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,COURSE_HISTORY,adj_ch,TOP_20
0,2023,2022-12-04,Hero World Challenge,Albany GC,Viktor Hovland,1,1,5.0,1.412,6.0,0.741,11.0,0.610,83.0,0.060,52.0,0.236,4.0,25.25%,15.0,3.00,3.0,3.94,27.0,4.56,4.0,80.0,43.0,307.6,37.0,63.70%,61.0,68.18%,47.0,62.10%,4.0,7.2840,16.0,93.3,1025.31,68.35,7,25.8,9.31,NaN,NaN,1
1,2023,2022-12-04,Hero World Challenge,Albany GC,Scottie Scheffler,2,2,1.0,2.615,1.0,1.021,1.0,1.194,5.0,0.399,161.0,-0.301,3.0,25.32%,12.0,2.99,1.0,3.91,2.0,4.46,6.0,88.0,31.0,310.3,57.0,62.09%,1.0,74.43%,8.0,64.94%,1.0,10.4741,7.5,87.5,2790.19,174.39,4,25.4,8.97,NaN,NaN,1
2,2023,2022-12-04,Hero World Challenge,Albany GC,Cameron Young,3,3,36.0,0.655,9.0,0.597,92.0,0.080,116.0,-0.007,153.0,-0.265,10.0,24.39%,25.0,3.01,88.0,4.02,14.0,4.54,14.0,113.0,5.0,316.5,108.0,58.54%,57.0,68.29%,154.0,56.84%,18.0,3.5932,18.0,76.9,1161.64,89.36,5,37.2,14.10,NaN,NaN,1
3,2023,2022-12-04,Hero World Challenge,Albany GC,Xander Schauffele,4,4,9.0,1.201,46.0,0.265,3.0,0.880,88.0,0.056,5.0,0.667,13.0,24.28%,25.0,3.01,6.0,3.96,14.0,4.54,98.0,196.0,69.0,304.1,127.0,57.46%,49.0,68.45%,32.0,63.04%,6.0,6.0742,10.0,84.6,1666.24,128.17,10,26.5,10.04,NaN,NaN,1
4,2023,2022-12-04,Hero World Challenge,Albany GC,Justin Thomas,5,5,17.0,0.998,60.0,0.198,41.0,0.362,4.0,0.436,131.0,-0.161,41.0,22.92%,159.0,3.10,118.0,4.03,14.0,4.54,81.0,185.0,38.0,308.5,147.0,55.19%,131.0,65.74%,111.0,58.78%,26.0,3.1021,12.0,92.9,1506.83,107.63,7,27.9,10.30,NaN,NaN,1
5,2023,2022-12-04,Hero World Challenge,Albany GC,Collin Morikawa,6,6,4.0,1.547,19.0,0.487,2.0,1.012,96.0,0.047,114.0,-0.109,15.0,24.01%,5.0,2.97,29.0,3.99,6.0,4.50,27.0,142.0,135.0,296.2,7.0,68.03%,5.0,70.99%,90.0,59.81%,13.0,4.0126,16.0,80.0,1063.16,70.88,5,40.6,14.64,NaN,NaN,1
6,2023,2022-12-04,Hero World Challenge,Albany GC,Tony Finau,7,7,7.0,1.257,42.0,0.288,5.0,0.789,50.0,0.178,122.0,-0.133,23.0,23.48%,94.0,3.05,20.0,3.98,11.0,4.52,32.0,147.0,65.0,304.2,82.0,60.70%,31.0,69.11%,62.0,61.24%,21.0,3.3486,9.0,82.4,2272.23,133.66,1,31.4,10.86,NaN,NaN,1
7,2023,2022-12-04,Hero World Challenge,Albany GC,Sungjae Im,T8,8,25.0,0.842,23.0,0.467,88.0,0.095,20.0,0.279,59.0,0.207,21.0,23.50%,59.0,3.03,29.0,3.99,4.0,4.47,46.0,160.0,111.0,299.3,49.0,62.98%,108.0,66.45%,40.0,62.58%,27.0,3.0549,14.0,93.8,1471.38,91.96,9,25.6,9.04,NaN,NaN,1
8,2023,2022-12-04,Hero World Challenge,Albany GC,Jon Rahm,T8,8,6.0,1.343,32.0,0.360,4.0,0.878,69.0,0.104,36.0,0.336,1.0,26.13%,2.0,2.95,6.0,3.96,2.0,4.46,23.0,135.0,11.0,314.0,124.0,57.68%,8.0,70.50%,57.0,61.54%,3.0,8.8349,5.0,100.0,1549.53,140.87,11,20.1,8.09,NaN,NaN,1
9,2023,2022-12-04,Hero World Challenge,Albany GC,Tom Kim,T10,10,26.0,0.834,73.0,0.128,10.0,0.655,92.0,0.050,72.0,0.115,14.0,24.14%,48.0,3.02,6.0,3.96,13.0,4.53,21.0,131.0,123.0,297.6,8.0,67.49%,24.0,69.51%,57.0,61.54%,11.0,4.2368,25.0,100.0,100.75,50.38,2,18.0,16.38,NaN,NaN,1


In [11]:
# Play with different years to see if I notice anything wrong (check Odds for tournament name mismatches)
training_df[training_df["SEASON"] == 2024].head()

,SEASON,ENDING_DATE,TOURNAMENT,COURSE,PLAYER,POS,FINAL_POS,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,COURSE_HISTORY,adj_ch,TOP_20
20,2024,2024-12-08,Hero World Challenge,Albany GC,Scottie Scheffler,1,1,1.0,2.401,2.0,0.816,1.0,1.269,17.0,0.316,75.0,0.095,1.0,27.93%,6.0,2.98,1.0,3.88,1.0,4.42,2.0,88.0,71.0,303.8,17.0,66.90%,2.0,73.16%,5.0,66.18%,1.0,16.5643,2.25,100.0,6224.750,478.83,13,5.7,2.16,1.5,1.37,1
21,2024,2024-12-08,Hero World Challenge,Albany GC,Tom Kim,2,2,49.0,0.452,73.0,0.132,52.0,0.263,83.0,0.057,115.0,-0.081,45.0,23.31%,99.0,3.05,85.0,4.01,101.0,4.59,45.0,147.0,112.0,299.0,35.0,64.92%,106.0,66.17%,124.0,58.00%,25.0,2.9110,20.00,73.7,943.182,49.64,0,46.8,15.62,10.0,14.43,1
22,2024,2024-12-08,Hero World Challenge,Albany GC,Justin Thomas,3,3,12.0,1.073,82.0,0.092,11.0,0.541,3.0,0.439,167.0,-0.499,43.0,23.47%,38.0,3.01,85.0,4.01,53.0,4.55,91.0,176.0,28.0,309.8,148.0,55.95%,142.0,64.73%,66.0,61.00%,32.0,2.6507,12.00,80.0,1214.175,80.94,6,38.2,13.78,4.0,3.64,1
23,2024,2024-12-08,Hero World Challenge,Albany GC,Akshay Bhatia,4,4,56.0,0.403,58.0,0.188,51.0,0.264,119.0,-0.048,34.0,0.345,87.0,22.35%,52.0,3.02,85.0,4.01,77.0,4.57,28.0,133.0,104.0,299.8,29.0,65.50%,114.0,66.02%,54.0,61.79%,29.0,2.7339,30.00,73.7,1862.456,98.02,2,40.4,13.49,NaN,NaN,1
24,2024,2024-12-08,Hero World Challenge,Albany GC,Keegan Bradley,5,5,46.0,0.482,62.0,0.174,39.0,0.301,100.0,0.007,127.0,-0.141,86.0,22.36%,142.0,3.08,107.0,4.02,77.0,4.57,15.0,116.0,56.0,305.5,60.0,62.55%,89.0,67.09%,144.0,56.41%,12.0,4.0174,35.00,81.2,689.182,43.07,4,45.3,15.99,13.0,18.76,1


### Normalization
***Fix Historical Odds***

Many names do not have historical odds but rather are part of the "field." So giving these "NaN" values the average odds (as I would for missing stats) isn't a good approximation of reality – they are typically the "field" because they are not notable and have poor odds individually.

There are also some crazy odds numbers occasionally (1000/1 or 3000/1) that don't happen all that much and are making that tail too long.

This cell assigns odds of 1000/1 for anyone missing and clips anything larger to 1000/1 to normalize the data better and assign the missing odds more appropriately where they belong.

***OWGR Adjustment***

The NaN's for OWGR are similarly bad players.  So we should assign these the worst ranking instead of the mean and clip it at 1000 just in case there are ever outliers.

For the OWGR score, it should match the lowest score in the dataset to assign that person the same as the worst.

***Recent Form Adjustment***

The NaN's for Recent Form mean that this player has not played any tournaments in the lookback period (9 months).  Similar to Odds and OWGR, I want to punish those that don't play often in my model.  Set these to 90 (i.e. like they miss a lot of cuts - because if they aren't playing they aren't good enough to make it to these tournaments very frequently).  The adj_form feature will also need to be updated here with the new data.

***FedEx Cup Point Adjustment***

The NaN's for FedEx Cup Points means there is no data.  Therefore they should be 0.

In [12]:
# === VEGAS ODDS ===
training_df["VEGAS_ODDS"] = training_df["VEGAS_ODDS"].fillna(1000).clip(upper=1000)

# === OWGR and OWGR_RANK ===
owgr_min = training_df["OWGR"].min(skipna=True)
training_df["OWGR"] = training_df["OWGR"].fillna(owgr_min)
training_df["OWGR_RANK"] = training_df["OWGR_RANK"].fillna(1000).astype(float).clip(upper=1000)

# === RECENT FORM and adj_form ===
training_df["RECENT_FORM"] = training_df["RECENT_FORM"].fillna(90)
if "TOTAL_EVENTS_PLAYED" in training_df.columns:
    training_df["adj_form"] = (
        training_df["RECENT_FORM"] / np.log1p(training_df["TOTAL_EVENTS_PLAYED"])
    ).round(2)

# === FEDEX CUP POINTS ===
training_df["FEDEX_CUP_POINTS"] = training_df["FEDEX_CUP_POINTS"].fillna(0)

training_df.head(5)


,SEASON,ENDING_DATE,TOURNAMENT,COURSE,PLAYER,POS,FINAL_POS,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,COURSE_HISTORY,adj_ch,TOP_20
0,2023,2022-12-04,Hero World Challenge,Albany GC,Viktor Hovland,1,1,5.0,1.412,6.0,0.741,11.0,0.610,83.0,0.060,52.0,0.236,4.0,25.25%,15.0,3.00,3.0,3.94,27.0,4.56,4.0,80.0,43.0,307.6,37.0,63.70%,61.0,68.18%,47.0,62.10%,4.0,7.2840,16.0,93.3,1025.31,68.35,7,25.8,9.31,NaN,NaN,1
1,2023,2022-12-04,Hero World Challenge,Albany GC,Scottie Scheffler,2,2,1.0,2.615,1.0,1.021,1.0,1.194,5.0,0.399,161.0,-0.301,3.0,25.32%,12.0,2.99,1.0,3.91,2.0,4.46,6.0,88.0,31.0,310.3,57.0,62.09%,1.0,74.43%,8.0,64.94%,1.0,10.4741,7.5,87.5,2790.19,174.39,4,25.4,8.97,NaN,NaN,1
2,2023,2022-12-04,Hero World Challenge,Albany GC,Cameron Young,3,3,36.0,0.655,9.0,0.597,92.0,0.080,116.0,-0.007,153.0,-0.265,10.0,24.39%,25.0,3.01,88.0,4.02,14.0,4.54,14.0,113.0,5.0,316.5,108.0,58.54%,57.0,68.29%,154.0,56.84%,18.0,3.5932,18.0,76.9,1161.64,89.36,5,37.2,14.10,NaN,NaN,1
3,2023,2022-12-04,Hero World Challenge,Albany GC,Xander Schauffele,4,4,9.0,1.201,46.0,0.265,3.0,0.880,88.0,0.056,5.0,0.667,13.0,24.28%,25.0,3.01,6.0,3.96,14.0,4.54,98.0,196.0,69.0,304.1,127.0,57.46%,49.0,68.45%,32.0,63.04%,6.0,6.0742,10.0,84.6,1666.24,128.17,10,26.5,10.04,NaN,NaN,1
4,2023,2022-12-04,Hero World Challenge,Albany GC,Justin Thomas,5,5,17.0,0.998,60.0,0.198,41.0,0.362,4.0,0.436,131.0,-0.161,41.0,22.92%,159.0,3.10,118.0,4.03,14.0,4.54,81.0,185.0,38.0,308.5,147.0,55.19%,131.0,65.74%,111.0,58.78%,26.0,3.1021,12.0,92.9,1506.83,107.63,7,27.9,10.30,NaN,NaN,1


#### Average the NaNs
At this point, the intentional offsetting of NaNs for poor players is over.  I don't want to penalize statistics or course history because you never know where they might land on the spectrum, so for these NaN values, we will take the mean.

In [13]:
# === Clean percentage stats stored as strings like '62.5%' ===
percent_stats = ["SCRAMBLING", "DRIVING_ACCURACY", "BIRDIES", "GIR"]

for col in percent_stats:
    if col in training_df.columns:
        training_df[col] = (
            training_df[col]
            .astype(str)
            .str.replace('%', '', regex=False)
            .replace(['None', 'nan', 'NaN', '--', 'DNP', ''], np.nan)
            .astype(float)
        )

# Select only the numeric columns
numeric_columns = training_df.select_dtypes(include=['float64', 'int64'])

# Fill NaN values with the mean of each column
numeric_columns = numeric_columns.fillna(numeric_columns.mean())

# Update the original dataframe with the filled numeric columns
training_df.update(numeric_columns)

training_df.head()

,SEASON,ENDING_DATE,TOURNAMENT,COURSE,PLAYER,POS,FINAL_POS,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,COURSE_HISTORY,adj_ch,TOP_20
0,2023,2022-12-04,Hero World Challenge,Albany GC,Viktor Hovland,1,1,5.0,1.412,6.0,0.741,11.0,0.610,83.0,0.060,52.0,0.236,4.0,25.25,15.0,3.00,3.0,3.94,27.0,4.56,4.0,80.0,43.0,307.6,37.0,63.70,61.0,68.18,47.0,62.10,4.0,7.2840,16.0,93.3,1025.31,68.35,7,25.8,9.31,9.409091,11.904545,1
1,2023,2022-12-04,Hero World Challenge,Albany GC,Scottie Scheffler,2,2,1.0,2.615,1.0,1.021,1.0,1.194,5.0,0.399,161.0,-0.301,3.0,25.32,12.0,2.99,1.0,3.91,2.0,4.46,6.0,88.0,31.0,310.3,57.0,62.09,1.0,74.43,8.0,64.94,1.0,10.4741,7.5,87.5,2790.19,174.39,4,25.4,8.97,9.409091,11.904545,1
2,2023,2022-12-04,Hero World Challenge,Albany GC,Cameron Young,3,3,36.0,0.655,9.0,0.597,92.0,0.080,116.0,-0.007,153.0,-0.265,10.0,24.39,25.0,3.01,88.0,4.02,14.0,4.54,14.0,113.0,5.0,316.5,108.0,58.54,57.0,68.29,154.0,56.84,18.0,3.5932,18.0,76.9,1161.64,89.36,5,37.2,14.10,9.409091,11.904545,1
3,2023,2022-12-04,Hero World Challenge,Albany GC,Xander Schauffele,4,4,9.0,1.201,46.0,0.265,3.0,0.880,88.0,0.056,5.0,0.667,13.0,24.28,25.0,3.01,6.0,3.96,14.0,4.54,98.0,196.0,69.0,304.1,127.0,57.46,49.0,68.45,32.0,63.04,6.0,6.0742,10.0,84.6,1666.24,128.17,10,26.5,10.04,9.409091,11.904545,1
4,2023,2022-12-04,Hero World Challenge,Albany GC,Justin Thomas,5,5,17.0,0.998,60.0,0.198,41.0,0.362,4.0,0.436,131.0,-0.161,41.0,22.92,159.0,3.10,118.0,4.03,14.0,4.54,81.0,185.0,38.0,308.5,147.0,55.19,131.0,65.74,111.0,58.78,26.0,3.1021,12.0,92.9,1506.83,107.63,7,27.9,10.30,9.409091,11.904545,1


In [14]:
# Check for NaN values
assert training_df.isna().sum().sum() == 0, "🚨 Still missing values!"

In [15]:
# Inspect which columns have NaN values
training_df.isna().sum().sort_values(ascending=False).head(10)


SEASON         0
ENDING_DATE    0
TOURNAMENT     0
COURSE         0
PLAYER         0
POS            0
FINAL_POS      0
SGTTG_RANK     0
SGTTG          0
SGOTT_RANK     0
dtype: int64

### Check Features for Outliers and General Health (Histograms)
Now that all the data is cleaned, let's look at the distribution graphs.

In [58]:
import plotly.express as px
import plotly.subplots as sp
import plotly.graph_objects as go
import pandas as pd
import numpy as np

# Select numeric columns with >1 unique value, excluding label
df_numeric = training_df.select_dtypes(include=np.number)
df_numeric = df_numeric.loc[:, df_numeric.nunique() > 1]
df_numeric = df_numeric.drop(columns=["TOP_20"], errors="ignore")

# Create subplots
n_cols = 3
n_rows = int(np.ceil(len(df_numeric.columns) / n_cols))
fig = sp.make_subplots(rows=n_rows, cols=n_cols, subplot_titles=df_numeric.columns)

# Add one histogram per feature with smart binning
for i, column in enumerate(df_numeric.columns):
    row = i // n_cols + 1
    col = i % n_cols + 1

    # Compute dynamic bin count
    col_range = df_numeric[column].max() - df_numeric[column].min()
    n_unique = df_numeric[column].nunique()
    nbins = min(50, max(5, int(n_unique / 2))) if col_range < 10 else 50  # tighter for short-range

    fig.add_trace(
        go.Histogram(
            x=df_numeric[column],
            nbinsx=nbins,
            histnorm="density",
            marker=dict(color="#636EFA"),
            opacity=0.8
        ),
        row=row,
        col=col
    )

# Style the figure
fig.update_layout(
    height=300 * n_rows,
    width=1000,
    title_text="Feature Distributions (with Adaptive Binning)",
    template="plotly_dark",
    showlegend=False
)

fig.show()

# Current Week Data

## Import 2025 Field
**Important!**  Save curren't week's `DKSalaries.csv` into Data folder before this step.

In [16]:
importlib.reload(utils.db_utils)
from utils.db_utils import DK_PLAYER_NAME_MAP
from utils.db_utils import standardize_player_names

# Load DraftKings player list
dk = pd.read_csv("data/DKSalaries.csv", usecols=["Name", "Salary"])

# Normalize DK names to PGA naming convention
dk["Name"] = dk["Name"].replace(DK_PLAYER_NAME_MAP)

# Rename for consistency
dk = dk.rename(columns={"Name": "PLAYER", "Salary": "SALARY"})

dk = standardize_player_names(dk)  # ✅ Normalize to match DB

dk.head(10)

,PLAYER,SALARY
0,Scottie Scheffler,14900
1,Robert MacIntyre,9800
2,J.J. Spaun,9500
3,Cameron Young,9200
4,Sam Burns,8900
5,Keegan Bradley,8700
6,Aaron Rai,8400
7,Hideki Matsuyama,8200
8,Justin Rose,8000
9,Alex Noren,7800


## Vegas Odds
Confirm the [Vegas Odds](http://golfodds.com/weekly-odds.html) page is updated with current tournament.

Note: sometimes this site can give errors and not load correctly.  Just have to do it again later.
Another trick to try is change the url try to run it and then put the url back the way it was.  Worked once, maybe random coincidence.

In [17]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_current_week_odds

odds_current = get_current_week_odds(season=2025, tournament_name=new_tournament_name)
odds_current.head(10)


,SEASON,TOURNAMENT,PLAYER,ODDS,VEGAS_ODDS
1,2025,Hero World Challenge,Scottie Scheffler,7/5,1.4
2,2025,Hero World Challenge,Robert MacIntyre,14/1,14.0
3,2025,Hero World Challenge,Cameron Young,14/1,14.0
4,2025,Hero World Challenge,J.J. Spaun,16/1,16.0
5,2025,Hero World Challenge,Keegan Bradley,18/1,18.0
6,2025,Hero World Challenge,Sam Burns,18/1,18.0
7,2025,Hero World Challenge,Hideki Matsuyama,18/1,18.0
8,2025,Hero World Challenge,Aaron Rai,20/1,20.0
9,2025,Hero World Challenge,Alex Noren,25/1,25.0
10,2025,Hero World Challenge,Justin Rose,18/1,18.0


## Cut Percentage and FedEx Points

In [18]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_cut_and_fedex_history

cuts_rolling = get_cut_and_fedex_history(db_path, this_week_history, window_months=9)
cuts_current = cuts_rolling[this_week_key].copy()
cuts_current.head(10)

,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,A.J. Ewart,1,0,0.000,0.0,0.00,0,2025-12-07,Hero World Challenge
1,Aaron Baddeley,9,2,4.950,22.2,0.55,1,2025-12-07,Hero World Challenge
2,Aaron Cockerill,1,0,0.000,0.0,0.00,0,2025-12-07,Hero World Challenge
3,Aaron Rai,15,12,716.355,80.0,47.76,6,2025-12-07,Hero World Challenge
4,Aaron Wise,6,1,6.333,16.7,1.06,0,2025-12-07,Hero World Challenge
5,Adam Hadwin,21,12,180.538,57.1,8.60,3,2025-12-07,Hero World Challenge
6,Adam Schenk,20,8,673.425,40.0,33.67,0,2025-12-07,Hero World Challenge
7,Adam Scott,15,12,356.438,80.0,23.76,3,2025-12-07,Hero World Challenge
8,Adam Svensson,18,7,126.650,38.9,7.04,0,2025-12-07,Hero World Challenge
9,Adrian Otaegui,1,0,0.000,0.0,0.00,0,2025-12-07,Hero World Challenge


## Recent Form

In [19]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_recent_avg_finish

recent_form = get_recent_avg_finish(db_path, this_week_history, window_months=9)
recent_form_current = recent_form[this_week_key].copy()

recent_form_current.head(10)

,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Scottie Scheffler,16,4.9,1.73,2025-12-07,Hero World Challenge
1,Jason Scrivener,1,8.0,11.54,2025-12-07,Hero World Challenge
2,Bubba Watson,1,14.0,20.20,2025-12-07,Hero World Challenge
3,Doc Redman,3,15.7,11.33,2025-12-07,Hero World Challenge
4,Jon Rahm,4,15.8,9.82,2025-12-07,Hero World Challenge
5,Andy Sullivan,1,17.0,24.53,2025-12-07,Hero World Challenge
6,Robin Williams,1,17.0,24.53,2025-12-07,Hero World Challenge
7,Rory McIlroy,12,17.7,6.90,2025-12-07,Hero World Challenge
8,Kazuki Higa,1,18.0,25.97,2025-12-07,Hero World Challenge
9,Ren Yonezawa,1,20.0,28.85,2025-12-07,Hero World Challenge


## Course History

In [20]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_course_history

course_hist = get_course_history(db_path, this_week_history, lookback_years=7)
course_hist_current = course_hist[this_week_key].copy()

course_hist_current.head(10)


,PLAYER,TOTAL_EVENTS_PLAYED,COURSE_HISTORY,adj_ch,ENDING_DATE,COURSE,TOURNAMENT
0,Aaron Rai,1,14.0,20.20,2025-12-07,Albany GC,Hero World Challenge
1,Akshay Bhatia,1,4.0,5.77,2025-12-07,Albany GC,Hero World Challenge
2,Billy Horschel,1,13.0,18.76,2025-12-07,Albany GC,Hero World Challenge
3,Brian Harman,2,10.0,9.10,2025-12-07,Albany GC,Hero World Challenge
4,Cameron Young,3,10.3,7.43,2025-12-07,Albany GC,Hero World Challenge
5,Collin Morikawa,2,6.5,5.92,2025-12-07,Albany GC,Hero World Challenge
6,Corey Conners,1,16.0,23.08,2025-12-07,Albany GC,Hero World Challenge
7,Jason Day,2,15.0,13.65,2025-12-07,Albany GC,Hero World Challenge
8,Jon Rahm,1,8.0,11.54,2025-12-07,Albany GC,Hero World Challenge
9,Jordan Spieth,2,10.5,9.56,2025-12-07,Albany GC,Hero World Challenge


## Merged Dataframe

In [21]:
importlib.reload(utils.db_utils)
from utils.db_utils import build_test_rows

this_week = build_test_rows(
    db_path=db_path,
    stats_df=stats_df,
    odds_df=odds_current,
    cuts_df=cuts_current,
    recent_form_df=recent_form_current,
    course_hist_df=course_hist_current,
    dk_df=dk,
    season=tournament_config["new"]["season"]
)

this_week.head()

,PLAYER,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,SEASON,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,COURSE_HISTORY,adj_ch,SALARY
0,Aaron Rai,12.0,0.942,22.0,0.391,21.0,0.476,68.0,0.077,132.0,-0.134,104.0,21.99%,179.0,3.16,16.0,3.97,28.0,4.54,74.0,173,171.0,289.6,2.0,73.85%,18.0,70.52%,101.0,59.16%,23,3.1098,2025.0,20.0,80.0,716.355,47.76,6.0,37.3,13.45,14.0,20.20,8400
1,Akshay Bhatia,57.0,0.361,103.0,0.024,14.0,0.575,162.0,-0.238,35.0,0.277,18.0,24.33%,10.0,2.99,42.0,3.99,145.0,4.65,155.0,226,132.0,298.7,94.0,60.51%,81.0,67.50%,153.0,56.19%,43,2.1282,2025.0,25.0,68.4,1088.592,57.29,0.0,46.8,15.62,4.0,5.77,7400
2,Alex Noren,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17,3.6870,2025.0,25.0,63.6,410.500,37.32,0.0,48.5,19.52,NaN,NaN,7800
3,Andrew Novak,84.0,0.162,102.0,0.042,113.0,-0.036,43.0,0.155,49.0,0.202,130.0,21.35%,146.0,3.09,152.0,4.05,36.0,4.55,162.0,240,102.0,301.3,138.0,57.20%,177.0,61.55%,93.0,59.42%,31,2.3854,2025.0,50.0,77.8,1249.292,69.41,3.0,40.5,13.75,NaN,NaN,6000
4,Billy Horschel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45,2.1093,2025.0,50.0,71.4,236.175,33.74,3.0,45.4,21.83,13.0,18.76,6500


### Dataframe Normalization

Run the same normalization techniques that I ran on the historical data:
- Any NaN Odds go to 1000/1 and clip all values at 1000/1
- OWGR NaN values go to 1000
- Recent form NaN values act like a MC at 90
- Average everything else.

In [22]:
# === 1. Clean percentage stats stored as strings like '62.5%' ===
percent_stats = ["SCRAMBLING", "DRIVING_ACCURACY", "BIRDIES", "GIR"]

for col in percent_stats:
    if col in this_week.columns:
        this_week[col] = (
            this_week[col]
            .astype(str)
            .str.replace('%', '', regex=False)
            .replace(['None', 'nan', 'NaN', '--', 'DNP', ''], np.nan)
        )
        this_week[col] = pd.to_numeric(this_week[col], errors="coerce")

# === 2. Normalize & assign fallback values for key fields ===
this_week["VEGAS_ODDS"] = pd.to_numeric(this_week["VEGAS_ODDS"], errors="coerce").fillna(1000).clip(upper=1000)

if "OWGR" in this_week.columns:
    this_week["OWGR"] = pd.to_numeric(this_week["OWGR"], errors="coerce")
    max_owgr = this_week["OWGR"].dropna().max()
    this_week["OWGR"] = this_week["OWGR"].fillna(max_owgr).clip(upper=1000)

if "OWGR_RANK" in this_week.columns:
    this_week["OWGR_RANK"] = pd.to_numeric(this_week["OWGR_RANK"], errors="coerce").fillna(1000).clip(upper=1000)

this_week["RECENT_FORM"] = pd.to_numeric(this_week["RECENT_FORM"], errors="coerce").fillna(90)
this_week["FEDEX_CUP_POINTS"] = pd.to_numeric(this_week["FEDEX_CUP_POINTS"], errors="coerce").fillna(0)

if "COURSE_HISTORY" in this_week.columns:
    this_week["COURSE_HISTORY"] = pd.to_numeric(this_week["COURSE_HISTORY"], errors="coerce")
    ch_mean = this_week["COURSE_HISTORY"].mean()
    this_week["COURSE_HISTORY"] = this_week["COURSE_HISTORY"].fillna(ch_mean)

# === 3. Fill all remaining NaNs in numeric columns ===
# Recast everything that might look numeric to be sure
for col in this_week.columns:
    if col not in ["PLAYER", "SALARY", "TOURNAMENT", "SEASON"]:
        try:
            this_week[col] = pd.to_numeric(this_week[col], errors="coerce")
        except Exception:
            continue

numeric_cols = this_week.select_dtypes(include=["number"]).columns.tolist()

for col in numeric_cols:
    if this_week[col].isna().any():
        col_mean = this_week[col].mean()
        this_week[col] = this_week[col].fillna(col_mean if not np.isnan(col_mean) else 0)


In [23]:
# === Final Check: No NaNs Should Remain ===
assert this_week.isna().sum().sum() == 0, "🚨 Still missing values in prediction set!"

In [24]:
# === Check for Remaining Missing Values ===
missing_summary = this_week.isna().sum()
missing_summary = missing_summary[missing_summary > 0].sort_values(ascending=False)

print("🧹 Columns still containing NaN values:")
display(missing_summary)

🧹 Columns still containing NaN values:


Series([], dtype: int64)

In [25]:
this_week.head(10)

,PLAYER,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,SEASON,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,COURSE_HISTORY,adj_ch,SALARY
0,Aaron Rai,12.000000,0.942000,22.000000,0.391000,21.000000,0.4760,68.000000,0.077000,132.000000,-0.134000,104.000000,21.990000,179.000000,3.160000,16.000000,3.970000,28.000000,4.540000,74.0,173.000000,171.000000,289.600000,2.000000,73.850000,18.000000,70.520000,101.000000,59.160000,23,3.1098,2025.0,20.0,80.0,716.355,47.76,6.0,37.3,13.45,14.00,20.200000,8400
1,Akshay Bhatia,57.000000,0.361000,103.000000,0.024000,14.000000,0.5750,162.000000,-0.238000,35.000000,0.277000,18.000000,24.330000,10.000000,2.990000,42.000000,3.990000,145.000000,4.650000,155.0,226.000000,132.000000,298.700000,94.000000,60.510000,81.000000,67.500000,153.000000,56.190000,43,2.1282,2025.0,25.0,68.4,1088.592,57.29,0.0,46.8,15.62,4.00,5.770000,7400
2,Alex Noren,50.222222,0.591222,61.222222,0.217222,63.611111,0.2805,72.611111,0.093611,60.666667,0.217056,65.833333,23.038333,73.611111,3.043889,60.388889,3.993333,59.055556,4.567778,85.5,178.277778,86.444444,303.661111,91.833333,60.963889,92.833333,67.091667,75.222222,60.626111,17,3.6870,2025.0,25.0,63.6,410.500,37.32,0.0,48.5,19.52,10.15,11.157143,7800
3,Andrew Novak,84.000000,0.162000,102.000000,0.042000,113.000000,-0.0360,43.000000,0.155000,49.000000,0.202000,130.000000,21.350000,146.000000,3.090000,152.000000,4.050000,36.000000,4.550000,162.0,240.000000,102.000000,301.300000,138.000000,57.200000,177.000000,61.550000,93.000000,59.420000,31,2.3854,2025.0,50.0,77.8,1249.292,69.41,3.0,40.5,13.75,10.15,11.157143,6000
4,Billy Horschel,50.222222,0.591222,61.222222,0.217222,63.611111,0.2805,72.611111,0.093611,60.666667,0.217056,65.833333,23.038333,73.611111,3.043889,60.388889,3.993333,59.055556,4.567778,85.5,178.277778,86.444444,303.661111,91.833333,60.963889,92.833333,67.091667,75.222222,60.626111,45,2.1093,2025.0,50.0,71.4,236.175,33.74,3.0,45.4,21.83,13.00,18.760000,6500
5,Brian Harman,81.000000,0.189000,112.000000,-0.015000,90.000000,0.0690,51.000000,0.134000,91.000000,0.039000,139.000000,21.100000,101.000000,3.060000,109.000000,4.020000,115.000000,4.620000,144.0,217.000000,163.000000,294.600000,54.000000,63.320000,131.000000,65.670000,66.000000,60.550000,33,2.3756,2025.0,35.0,81.2,1570.836,98.18,0.0,41.9,14.79,10.00,9.100000,6400
6,Cameron Young,86.000000,0.160000,31.000000,0.317000,129.000000,-0.1240,107.000000,-0.033000,7.000000,0.642000,22.000000,24.140000,131.000000,3.080000,69.000000,4.000000,51.000000,4.570000,97.0,184.000000,17.000000,313.200000,167.000000,53.850000,159.000000,64.180000,95.000000,59.360000,18,3.5257,2025.0,14.0,77.8,1963.557,109.09,4.0,39.1,13.28,10.30,7.430000,9200
7,Chris Gotterup,36.000000,0.581000,9.000000,0.566000,105.000000,-0.0130,82.000000,0.029000,101.000000,0.004000,48.000000,23.140000,37.000000,3.020000,10.000000,3.960000,36.000000,4.550000,54.0,161.000000,7.000000,316.700000,154.000000,55.750000,22.000000,70.330000,19.000000,63.580000,28,2.7968,2025.0,30.0,75.0,1253.313,78.33,9.0,39.5,13.94,10.15,11.157143,7100
8,Corey Conners,39.000000,0.550000,24.000000,0.388000,39.000000,0.3150,146.000000,-0.153000,67.000000,0.133000,97.000000,22.150000,101.000000,3.060000,42.000000,3.990000,101.000000,4.610000,52.0,160.000000,137.000000,298.200000,23.000000,66.640000,44.000000,69.200000,109.000000,59.020000,30,2.6416,2025.0,30.0,93.3,1404.952,93.66,4.0,27.3,9.85,16.00,23.080000,6800
9,Harris English,80.000000,0.193000,47.000000,0.260000,119.000000,-0.0700,89.000000,0.003000,18.000000,0.418000,166.000000,20.200000,64.000000,3.040000,69.000000,4.000000,83.000000,4.600000,44.0,157.000000,83.000000,304.300000,74.000000,61.760000,104.000000,66.67

## Correlations
**Last 4 Years**

In [26]:
import plotly.express as px

# Get the 4 most recent seasons (or fewer if not available)
available_seasons = sorted(training_df["SEASON"].unique())[-4:]

# Dictionary to store correlation results
correlation_frames = {}

# Calculate correlations for each season
for season in available_seasons:
    df = training_df[training_df["SEASON"] == season].copy()

    # Numeric features only, drop final result
    numeric = df.select_dtypes(include=["float64", "int64"]).drop(columns=["FINAL_POS", "SEASON"], errors="ignore")

    # Skip if TOP_20 is not present
    if "TOP_20" not in numeric.columns:
        continue

    # Correlation with TOP_20
    cor = numeric.corr().abs()["TOP_20"].drop("TOP_20", errors="ignore").sort_values(ascending=False)
    cor_df = cor.reset_index()
    cor_df.columns = ["Feature", "Correlation"]
    correlation_frames[season] = cor_df

# Plot one bar chart per season
for season, cor_df in correlation_frames.items():
    fig = px.bar(
        cor_df,
        x="Feature",
        y="Correlation",
        title=f"🔎 Correlation to TOP_20 — Season {int(season)}",
        height=500,
        width=1000
    )

    fig.update_layout(
        xaxis_title="Feature",
        yaxis_title="Absolute Correlation to TOP_20",
        template="plotly_dark",
        xaxis_tickangle=-45
    )

    fig.show()


**Average of Last 4 Years**

In [27]:
import pandas as pd
import numpy as np
import plotly.express as px

# === Rebuild correlation matrices for the 4 most recent seasons ===
cor = {}
recent_seasons = sorted(training_df["SEASON"].unique())[-4:]

for season in recent_seasons:
    df_season = training_df[training_df["SEASON"] == season].copy()
    numeric_cols = df_season.select_dtypes(include=["float64", "int64"])
    cor[int(season)] = numeric_cols.corr().abs()

# === Combine into one summary DataFrame ===
cor_df = pd.DataFrame()

for season in recent_seasons:
    df = cor[season].reset_index()
    if "TOP_20" not in df.columns:
        continue
    temp = df[["index", "TOP_20"]].copy()
    temp = temp.rename(columns={"TOP_20": str(season)})
    if cor_df.empty:
        cor_df = temp
    else:
        cor_df = cor_df.merge(temp, on="index", how="outer")

# Drop target and metadata rows
cor_df = cor_df[~cor_df["index"].isin(["TOP_20", "FINAL_POS", "SEASON"])]

# Compute average correlation across seasons
cor_df["AVERAGE"] = cor_df.drop(columns=["index"]).mean(axis=1)
cor_df = cor_df.sort_values(by="AVERAGE", ascending=False)

# === Plotly Bar Chart ===
fig = px.bar(
    cor_df,
    x="index",
    y="AVERAGE",
    title="Average Correlation to TOP_20 (Last 4 Seasons)",
    labels={"index": "Feature", "AVERAGE": "Avg Correlation"},
    template="plotly_dark"
)

fig.update_layout(
    xaxis_tickangle=-45,
    height=600,
    width=1000,
    showlegend=False
)

fig.show()


# ML Model

## Predictors

In [28]:
# === Define Features and Target ===
target_col = "TOP_20"

# Columns to exclude from features
exclude = [
    "PLAYER", "TOURNAMENT", "COURSE", "ENDING_DATE", "SEASON", "TOURN_ID",
    "TOP_20", "FINAL_POS"  
]

# Select numeric feature columns
feature_cols = [
    col for col in training_df.columns
    if col not in exclude and training_df[col].dtype in [np.float64, np.int64]
]

print(f"Selected {len(feature_cols)} features:\n", feature_cols)

Selected 39 features:
 ['SGTTG_RANK', 'SGTTG', 'SGOTT_RANK', 'SGOTT', 'SGAPR_RANK', 'SGAPR', 'SGATG_RANK', 'SGATG', 'SGP_RANK', 'SGP', 'BIRDIES_RANK', 'BIRDIES', 'PAR_3_RANK', 'PAR_3', 'PAR_4_RANK', 'PAR_4', 'PAR_5_RANK', 'PAR_5', 'TOTAL_DRIVING_RANK', 'TOTAL_DRIVING', 'DRIVING_DISTANCE_RANK', 'DRIVING_DISTANCE', 'DRIVING_ACCURACY_RANK', 'DRIVING_ACCURACY', 'GIR_RANK', 'GIR', 'SCRAMBLING_RANK', 'SCRAMBLING', 'OWGR_RANK', 'OWGR', 'VEGAS_ODDS', 'CUT_PERCENTAGE', 'FEDEX_CUP_POINTS', 'form_density', 'CONSECUTIVE_CUTS', 'RECENT_FORM', 'adj_form', 'COURSE_HISTORY', 'adj_ch']


### Random Forest
Initial default model before hyperparameter tuning.

This one actually performed really well (ROC AUC above 0.85).  Strangely doing hyperparameter tuning hurt the model (less than 0.7), so I removed that step and we will just use this default model.  This does cross-validation testing on 5 folds to ensure robust train/test splits.

In [29]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

# === Standardize features ===
scaler = StandardScaler()
X_rf_std = scaler.fit_transform(training_df[feature_cols])
y_rf = training_df["TOP_20"]

# === Balance the dataset with SMOTE and undersampling ===
over = SMOTE(sampling_strategy=0.5, k_neighbors=3)
under = RandomUnderSampler(sampling_strategy=0.5)
pipeline = Pipeline([("o", over), ("u", under)])
X_rf_bal, y_rf_bal = pipeline.fit_resample(X_rf_std, y_rf)

# === Define and evaluate Random Forest ===
rf = RandomForestClassifier(n_estimators=100, random_state=42)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(rf, X_rf_bal, y_rf_bal, cv=cv, scoring="roc_auc")

print("📊 Random Forest (All Features)")
print("Cross-validation scores:", scores)
print("Average ROC AUC score: {:.2f}".format(scores.mean()))


ValueError: The target 'y' needs to have more than 1 class. Got 1 class instead

Random Forest Training & Feature Importance

Now we train on all the data (no longer cross-validation folds) and extract the feature importance to see how it compares with linear correlations.


In [74]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.preprocessing import StandardScaler
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
import pandas as pd
import plotly.express as px

# === Resample the full dataset ===
over = SMOTE(sampling_strategy=0.5, k_neighbors=3)
under = RandomUnderSampler(sampling_strategy=0.5)
resample_pipeline = Pipeline([("o", over), ("u", under)])
X_all_resampled, y_all_resampled = resample_pipeline.fit_resample(
    training_df[feature_cols], training_df["TOP_20"]
)

# === Standardize ===
scaler = StandardScaler()
X_all_scaled = scaler.fit_transform(X_all_resampled)

# === Train Calibrated Random Forest ===
base_rf = RandomForestClassifier(random_state=42)
final_model = CalibratedClassifierCV(estimator=base_rf, method="sigmoid", cv=5)
final_model.fit(X_all_scaled, y_all_resampled)

# === Aggregate feature importances across all CV folds ===
all_importances = np.array([
    est.estimator.feature_importances_ for est in final_model.calibrated_classifiers_
])
mean_importance = all_importances.mean(axis=0)

importance_df = pd.DataFrame({
    "Feature": feature_cols,
    "Importance": mean_importance
}).sort_values(by="Importance", ascending=False)

# === Plot feature importances ===
fig = px.bar(
    importance_df,
    x="Feature",
    y="Importance",
    title="Random Forest Feature Importances",
    template="plotly_dark"
)
fig.update_layout(xaxis_tickangle=-45)
fig.show()


Use this model on this week's data to predict the best performers.

In [75]:
# === Prepare test data (this week's players) ===

# Select and standardize features using the same scaler as training
X_test = this_week[feature_cols]
X_test_scaled = scaler.transform(X_test)

# === Predict probabilities for Top 20 finish ===
# CalibratedClassifierCV returns well-calibrated probabilities
this_week["PROBABILITY"] = final_model.predict_proba(X_test_scaled)[:, 1]

# === Sort results by highest predicted probability ===
this_week_sorted = this_week.sort_values(by="PROBABILITY", ascending=False).reset_index(drop=True)

# === Define final column order explicitly ===
columns_to_show = [
    "PLAYER", "SALARY", "PROBABILITY",
    "RECENT_FORM", "COURSE_HISTORY", "VEGAS_ODDS", "CUT_PERCENTAGE", 
    "OWGR_RANK", "FEDEX_CUP_POINTS",

    # SG stats
    "SGTTG_RANK", "SGOTT_RANK", "SGAPR_RANK", "SGATG_RANK", "SGP_RANK",

    # Scoring/Performance
    "BIRDIES_RANK", "PAR_3_RANK", "PAR_4_RANK", "PAR_5_RANK",

    # Driving / Ball Striking
    "TOTAL_DRIVING_RANK", "DRIVING_DISTANCE_RANK", "DRIVING_ACCURACY_RANK",
    "GIR_RANK", "SCRAMBLING_RANK"
]

# Keep only columns that exist in the dataframe
columns_to_show = [col for col in columns_to_show if col in this_week_sorted.columns]

# Final dataframe for display/export
export_df = this_week_sorted[columns_to_show].copy()

# Preview top 20
export_df.head(20)


,PLAYER,SALARY,PROBABILITY,RECENT_FORM,COURSE_HISTORY,VEGAS_ODDS,CUT_PERCENTAGE,OWGR_RANK,FEDEX_CUP_POINTS,SGTTG_RANK,SGOTT_RANK,SGAPR_RANK,SGATG_RANK,SGP_RANK,BIRDIES_RANK,PAR_3_RANK,PAR_4_RANK,PAR_5_RANK,TOTAL_DRIVING_RANK,DRIVING_DISTANCE_RANK,DRIVING_ACCURACY_RANK,GIR_RANK,SCRAMBLING_RANK
0,Jacob Bridgeman,7900,0.947764,48.2,72.000000,60.0,69.6,69.0,1393.989,123.000000,125.0,125.000000,59.000000,18.000000,62.000000,34.000000,81.000000,114.000000,110.000000,95.000000,94.000000,146.000000,28.000000
1,Lee Hodges,7100,0.916152,58.4,50.700000,100.0,58.8,151.0,251.200,38.000000,78.0,29.000000,79.000000,99.000000,76.000000,34.000000,23.000000,33.000000,43.000000,118.000000,33.000000,16.000000,21.000000
2,Doug Ghim,7400,0.868698,50.7,51.000000,65.0,68.4,156.0,353.849,13.000000,43.0,17.000000,54.000000,176.000000,103.000000,62.000000,57.000000,59.000000,28.000000,91.000000,41.000000,20.000000,98.000000
3,Mac Meissner,7500,0.793211,51.4,59.050505,60.0,66.7,104.0,565.714,52.000000,140.0,37.000000,24.000000,115.000000,129.000000,34.000000,57.000000,127.000000,106.000000,83.000000,102.000000,95.000000,8.000000
4,Vince Whaley,8700,0.777363,46.9,47.800000,35.0,75.0,114.0,564.108,86.000000,101.0,115.000000,38.000000,34.000000,64.000000,2.000000,57.000000,11.000000,121.000000,48.000000,148.000000,60.000000,7.000000
5,Alex Smalley,9000,0.696225,55.4,57.300000,35.0,50.0,134.0,558.756,34.000000,18.0,89.000000,61.000000,77.000000,23.000000,107.000000,4.000000,48.000000,12.000000,61.000000,46.000000,56.000000,26.000000
6,Kevin Roy,7100,0.678862,49.6,68.000000,100.0,65.0,139.0,513.037,59.000000,39.0,96.000000,86.000000,63.000000,57.000000,81.000000,8.000000,41.000000,9.000000,63.000000,37.000000,48.000000,49.000000
7,Jesper Svensson,7500,0.674506,45.5,59.050505,70.0,78.9,137.0,405.028,46.000000,9.0,138.000000,66.000000,89.000000,12.000000,107.000000,57.000000,9.000000,38.000000,3.000000,143.000000,79.000000,126.000000
8,Bud Cauley,8000,0.642066,40.2,25.000000,50.0,82.4,65.0,1064.164,33.000000,57.0,31.000000,95.000000,67.000000,114.000000,107.000000,81.000000,68.000000,106.000000,101.000000,84.000000,120.000000,67.000000
9,Matt Kuchar,8300,0.619062,39.9,66.200000,50.0,86.7,129.0,414.183,92.000000,152.0,74.000000,25.000000,15.000000,69.000000,62.000000,4.000000,143.000000,136.000000,176.000000,34.000000,122.000000,4.000000


### Save to CSV

In [76]:
# Ensure required columns for spreadsheet export
if "COURSE_HISTORY" not in export_df.columns:
    export_df["COURSE_HISTORY"] = "-"

# Define columns to round (FedEx + stats)
columns_to_round = [
    "FEDEX_CUP_POINTS",
    "SGTTG_RANK", "SGOTT_RANK", "SGAPR_RANK", "SGATG_RANK", "SGP_RANK",
    "BIRDIES_RANK", "PAR_3_RANK", "PAR_4_RANK", "PAR_5_RANK",
    "TOTAL_DRIVING_RANK", "DRIVING_DISTANCE_RANK", "DRIVING_ACCURACY_RANK",
    "GIR_RANK", "SCRAMBLING_RANK"
]

columns_to_round_1 = [
    "COURSE_HISTORY", "CUT_PERCENTAGE"
]

# Round only the selected columns to 0 decimal places
export_df[columns_to_round] = export_df[columns_to_round].round(0).astype("Int64")
# Round only the selected columns to 1 decimal places
export_df[columns_to_round_1] = export_df[columns_to_round_1].round(1)

# Explicit column order for Excel integration
column_order = [
    "PLAYER", "SALARY", "PROBABILITY", "RECENT_FORM", "COURSE_HISTORY", "VEGAS_ODDS", 
    "CUT_PERCENTAGE", "OWGR_RANK", "FEDEX_CUP_POINTS", 
    "SGTTG_RANK", "SGOTT_RANK", "SGAPR_RANK", "SGATG_RANK", "SGP_RANK",
    "BIRDIES_RANK", "PAR_3_RANK", "PAR_4_RANK", "PAR_5_RANK",
    "TOTAL_DRIVING_RANK", "DRIVING_DISTANCE_RANK", "DRIVING_ACCURACY_RANK",
    "GIR_RANK", "SCRAMBLING_RANK"
]

# Ensure all expected columns exist
for col in column_order:
    if col not in export_df.columns:
        export_df[col] = "-" if col == "COURSE_HISTORY" else np.nan

# Reorder columns to match Excel layout
export_df = export_df[column_order]

# Export to CSV
filename = f"data/current_week_export.csv"
export_df.to_csv(filename, index=False)
print(f"✅ Exported to {filename}")

✅ Exported to data/current_week_export.csv
